# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902055


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:04<01:04,  2.15s/it]

Rendering models:   9%|▉         | 3/32 [00:05<00:57,  1.97s/it]

Rendering models:  12%|█▎        | 4/32 [00:06<00:40,  1.45s/it]

Rendering models:  16%|█▌        | 5/32 [00:06<00:30,  1.15s/it]

...

Rendering models:  88%|████████▊ | 28/32 [00:19<00:01,  2.05it/s]

Rendering models:  91%|█████████ | 29/32 [00:20<00:01,  1.86it/s]

Rendering models:  94%|█████████▍| 30/32 [00:21<00:01,  1.79it/s]

Rendering models:  97%|█████████▋| 31/32 [00:21<00:00,  1.75it/s]

equidad1                                    0.044687
not-logged-in-cae133427d99aac1e554          0.000687
kayleebug2017                               0.000803
JessieDawn                                  0.001124
not-logged-in-0b0736c8991a7cc2017a          0.000539
I_wish_I_can_study_at_Starfleet_Academy     0.001524
ElisabethB                                  0.014787
Thedunkmasta                                0.002498
OregonHiker                                 0.000505
not-logged-in-1be611b9f88e48053369          0.001072
not-logged-in-f46d44665a162c208faf          0.006841
Jboy2000                                    0.010054
earthling001                                0.000773
not-logged-in-553ff4a3bbd18b109e47          0.000762
jmartinez468                                0.033142
CODGOD874                                  10.177697
withadnotat                                 0.000469
acapirala                                   0.005988
Dpwillet                                    0.

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())